In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/Src/utils")

In [2]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# function computing initial conditions (depending on \mu)
from ipynb.fs.defs.init_level_set import _init_level_set

# geodesics differential equation
from ipynb.fs.defs.geodesic_equation import _geodesic_equation

# find closed curve
from ipynb.fs.defs.closed_curve import _closed_curve

# solve_ivp is used for integration
from scipy.integrate import solve_ivp

# tqdm shows progress bar
from tqdm.notebook import tqdm

# library for parallel computing
from joblib import Parallel, delayed

# library for Polygon, Point objects
from shapely.geometry import Polygon, Point

# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [3]:
def _closed_null_geodesics(X, Y, mu, interp_phi_prime, d_threshold, S11, interp_DOE, density_ic, Ncores):
    
    defined_domain = np.isfinite(S11).astype(int)
    
    # compute initial conditions
    x0mu, y0mu, phi0mu = _init_level_set(X, Y, S11, mu, density_ic)
    
    # define integration domain of dummy variable
    t = [0, 10]
    
    # define resolution of trajectories
    t_eval = np.linspace(t[0], t[1], 10000)
    
    # function which launches geodesic trajectories from initial condition y0 = [x0lam[j], y0lam[j], phi0lam[j]] (with parallel computing)
    def launch_trajectories(j):
        
        # initial condition
        y0 = [x0mu[j], y0mu[j], phi0mu[j]]
        
        # integrate trajectory from initial condition
        sol = solve_ivp(_geodesic_equation, t, y0, 'RK45', t_eval, rtol=1e-05, atol=1e-05, first_step = 0.01, args=(interp_phi_prime, X, Y, defined_domain, interp_DOE))
    
        # store x, y, phi 
        x = sol.y[0,:]
        y = sol.y[1,:]
        phi = sol.y[2,:]
        
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance
        x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic = _closed_curve(x, y, phi, np.array(y0), d_threshold)
        
        return x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic
    
    # compute closed null-geodesics
    solODE_closed_curves = Parallel(n_jobs=Ncores, verbose = 0)(delayed(launch_trajectories)(j) for j in tqdm(range(len(x0mu))))
    
    print("Finished integration of closed null geodesics")
    
    return solODE_closed_curves, [x0mu, y0mu]